In [4]:
!pip install graphviz
!pip install web3

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 47 kB 1.9 MB/s             


In [2]:
from tree import Node, insert, vis

root = Node(1)
for key in range(2, 2**5):
    insert(root, key)

vis(root)

In [4]:
from pprint import pprint
import json

from lib import write_8_bits, keccak256_str
from game_types import VMStatus
from block import Block
from game import (
    FaultDisputeGame, 
    VM,
    ABSOLUTE_PRESTATE,
    MAX_GAME_DEPTH,
    SPLIT_DEPTH, MAX_CLOCK_DURATION,
    CLOCK_EXTENSION,
    CHALLENGE_PERIOD
)

block = Block()
vm = VM()
game = FaultDisputeGame(vm = vm)
game.initialize(
    l2_root = keccak256_str("l2 root"),
    l2_block_num = 1,
    root_claim = keccak256_str("root"),
    game_creator = "defender",
    bond = 0,
    block_timestamp = block.timestamp
)

args = [{
    "challenge_idx": 0,
    "disputed": keccak256_str("root"), 
    "claim": keccak256_str("2,0"), 
    "is_attack": True, 
    "msg_sender": "challenger"
}, {
    "challenge_idx": 1,
    "disputed": keccak256_str("2,0"), 
    "claim": keccak256_str("4,0"), 
    "is_attack": True, 
    "msg_sender": "defender"
}, {
    "challenge_idx": 2,
    "disputed": keccak256_str("4,0"), 
    "claim": write_8_bits(keccak256_str("8,0"), VMStatus.INVALID.value), 
    "is_attack": True, 
    "msg_sender": "challenger"
}, {
    "challenge_idx": 3,
    "disputed": write_8_bits(keccak256_str("8,0"), VMStatus.INVALID.value),
    "claim": keccak256_str("16,0"),
    "is_attack": True, 
    "msg_sender": "defender"
}]

for arg in args:
    block.inc()
    game.move(
        disputed = arg["disputed"], 
        challenge_idx = arg["challenge_idx"], 
        claim = arg["claim"], 
        is_attack = arg["is_attack"], 
        msg_value = 0, 
        block_timestamp = block.timestamp,
        msg_sender = arg["msg_sender"],
    )

### Attack ###
# block.inc()
# game.step(
#     claim_idx = 4, 
#     is_attack = True, 
#     state_data = "0x00", 
#     proof = "",
#     block_timestamp = block.timestamp,
#     msg_sender = "challenger",
# )

### Defend, parent post disagree ###
# vm.set_post_state(write_8_bits(keccak256_str("8,0"), VMStatus.INVALID.value))
# block.inc()
# game.step(
#     claim_idx = 4, 
#     is_attack = False, 
#     state_data = "16,0", 
#     proof = "",
#     block_timestamp = block.timestamp,
#     msg_sender = "challenger",
# )

block.warp(MAX_CLOCK_DURATION + 2)
game.resolve_claim(4, 0, block_timestamp = block.timestamp)
game.resolve_claim(3, 0, block_timestamp = block.timestamp)
game.resolve_claim(2, 0, block_timestamp = block.timestamp)
game.resolve_claim(1, 0, block_timestamp = block.timestamp)
game.resolve_claim(0, 0, block_timestamp = block.timestamp)
game.resolve()
pprint(game.status)

pprint(game.resolution_checkpoints)
pprint(game.claim_data)

TypeError: exceptions must derive from BaseException

In [ ]:

###  no challenge -> defender wins ###
# block.warp(MAX_CLOCK_DURATION)
# game.resolve_claim(0, 0, block_timestamp = block.timestamp
# game.resolve()
# game.status

### attack -> challenger wins ###
# block.inc()
# game.move(
#     disputed = keccak256_str("root"), 
#     challenge_idx = 0, 
#     claim = keccak256_str("2,0"), 
#     is_attack = True, 
#     msg_value = 0, 
#     block_timestamp = block.timestamp,
#     msg_sender = "challenger",
# )

# # Each party (defender and challenger) gets MAX_CLOCK_DURATION
# # Need to wait for both parties to use MAX_CLOCK_DURATION
# block.warp(MAX_CLOCK_DURATION + 1)
# game.resolve_claim(1, 0, block_timestamp = block.timestamp)
# game.resolve_claim(0, 0, block_timestamp = block.timestamp)
# game.resolve()
# game.status

### attack -> defend -> defender wins ###
# block.inc()
# game.move(
#     disputed = keccak256_str("root"), 
#     challenge_idx = 0, 
#     claim = keccak256_str("2,0"), 
#     is_attack = True, 
#     msg_value = 0, 
#     block_timestamp = block.timestamp,
#     msg_sender = "challenger",
# )

# block.inc()
# game.move(
#     disputed = keccak256_str("2,0"), 
#     challenge_idx = 1, 
#     claim = keccak256_str("6,0"), 
#     is_attack = False, 
#     msg_value = 0, 
#     block_timestamp = block.timestamp,
#     msg_sender = "defender",
# )

# block.warp(MAX_CLOCK_DURATION + 1)
# game.resolve_claim(2, 0, block_timestamp = block.timestamp)
# game.resolve_claim(1, 0, block_timestamp = block.timestamp)
# game.resolve_claim(0, 0, block_timestamp = block.timestamp)
# game.resolve()
# game.status

### attack -> attack -> defender wins ###
# block.inc()
# game.move(
#     disputed = keccak256_str("root"), 
#     challenge_idx = 0, 
#     claim = keccak256_str("2,0"), 
#     is_attack = True, 
#     msg_value = 0, 
#     block_timestamp = block.timestamp,
#     msg_sender = "challenger",
# )

# block.inc()
# game.move(
#     disputed = hash_str("2,0"), 
#     challenge_idx = 1, 
#     claim = hash_str("4,0"), 
#     is_attack = True, 
#     msg_value = 0, 
#     block_timestamp = block.timestamp,
#     msg_sender = "defender",
# )

# block.warp(MAX_CLOCK_DURATION + 1)
# game.resolve_claim(2, 0, block_timestamp = block.timestamp)
# game.resolve_claim(1, 0, block_timestamp = block.timestamp)
# game.resolve_claim(0, 0, block_timestamp = block.timestamp)
# game.resolve()
# game.status

### attack -> defend -> defend -> challenger wins ###
block.inc()
game.move(
    disputed = hash_str("root"), 
    challenge_idx = 0, 
    claim = hash_str("2,0"), 
    is_attack = True, 
    msg_value = 0, 
    block_timestamp = block.timestamp,
    msg_sender = "challenger",
)

block.inc()
game.move(
    disputed = hash_str("2,0"), 
    challenge_idx = 1, 
    claim = hash_str("6,0"), 
    is_attack = False, 
    msg_value = 0, 
    block_timestamp = block.timestamp,
    msg_sender = "defender",
)

block.inc()
game.move(
    disputed = hash_str("6,0"), 
    challenge_idx = 2, 
    claim = hash_str("14,0"), 
    is_attack = False, 
    msg_value = 0, 
    block_timestamp = block.timestamp,
    msg_sender = "challenger",
)

block.warp(MAX_CLOCK_DURATION + 2)
game.resolve_claim(3, 0, block_timestamp = block.timestamp)
game.resolve_claim(2, 0, block_timestamp = block.timestamp)
game.resolve_claim(1, 0, block_timestamp = block.timestamp)
game.resolve_claim(0, 0, block_timestamp = block.timestamp)
game.resolve()
game.status